In [3]:
lst = [1,2,3];
lst.index(1)

0

In [4]:
import gym
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten
from keras.optimizers import Adam
from keras.layers.convolutional import Convolution2D


class PGAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.99
        self.learning_rate = 0.001
        self.states = []
        self.gradients = []
        self.rewards = []
        self.probs = []
        self.model = self._build_model()
        self.summary = self.model.summary;

    def _build_model(self):
        model = Sequential()
        model.add(Reshape((80, 80, 1), input_shape=(self.state_size,)))
        model.add(Convolution2D(25, (6, 6), subsample=(3, 3), border_mode='same',
                                activation='relu', init='he_uniform'))
        model.add(Convolution2D(5, (6, 6), subsample=(1, 1), border_mode='same',
                                activation='relu', init='he_uniform'))
        model.add(Flatten())
        model.add(Dense(20, activation='relu', init='he_uniform'))
        model.add(Dense(20, activation='relu', init='he_uniform'))
        model.add(Dense(self.action_size, activation='softmax'))
        opt = Adam(lr=self.learning_rate)
        model.compile(loss='categorical_crossentropy', optimizer=opt)
        return model

    def remember(self, state, action, prob, reward):
        y = np.zeros([self.action_size])
        y[action] = 1
        self.gradients.append(np.array(y).astype('float32') - prob)
        self.states.append(state)
        self.rewards.append(reward)

    def act(self, state):
        # state = state.reshape([1, state.shape[0]])
        aprob = self.model.predict(state, batch_size=1).flatten()
        self.probs.append(aprob)
        prob = aprob / np.sum(aprob)
        action = np.random.choice(self.action_size, 1, p=prob)[0]
        return action, prob

    def discount_rewards(self, rewards):
        discounted_rewards = np.zeros_like(rewards)
        running_add = 0
        for t in reversed(range(0, rewards.size)):
            if rewards[t] != 0:
                running_add = 0
            running_add = running_add * self.gamma + rewards[t]
            discounted_rewards[t] = running_add
        return discounted_rewards

    def train(self,rewards):
        gradients = np.vstack(self.gradients)
        # rewards = np.vstack(self.rewards)
        # rewards = self.discount_rewards(rewards)
        rewards = (rewards - np.mean(rewards,keepdims=1)) / np.std(rewards)
        gradients *= rewards
        X = np.squeeze(np.vstack([self.states]))
        Y = self.probs + self.learning_rate * np.squeeze(np.vstack([gradients]))
        self.model.train_on_batch(X, Y)
        self.states, self.probs, self.gradients, self.rewards = [], [], [], []

    def load(self, name):
        global episode
        self.model.load_weights(name)
    def readlog(self, LogName):
#         LogName = self.LogName;
        with open(LogName,'rb') as f:
                first = f.readline()      # Read the first line.
                f.seek(-2, 2)             # Jump to the second last byte.
                while f.read(1) != b"\n": # Until EOL is found...
                    f.seek(-2, 1)         # ...jump back the read byte plus one more.
                last = f.readline() 
                lst = last.split('\t');
                eind = lst.index('Episode')+1;
                self.episode = int(lst[eind]);

    def save(self, name):
        self.model.save_weights(name)

def preprocess(I):
    I = I[35:195]
    I = I[::2, ::2, 0]
    I[I == 144] = 0
    I[I == 109] = 0
    I[I != 0] = 1
    return I.astype(np.float).ravel()
if __name__ == "__main__":
    env = gym.make("Pong-v0")
    state = env.reset()
    prev_x = None
    score = 0
    episode = 0

    state_size = 80 * 80
    action_size = env.action_space.n
    agent = PGAgent(state_size, action_size)
    
agent.summary();

Using TensorFlow backend.
[2017-06-13 13:55:55,343] Making new env: Pong-v0


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 80, 80, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 25)        925       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 5)         4505      
_________________________________________________________________
flatten_1 (Flatten)          (None, 3645)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                72920     
_________________________________________________________________
dense_2 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 126       
Total para

/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(25, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(3, 3))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(1, 1))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:30: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="he_uniform", activation="relu")`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="he_uniform", activation="relu")`


In [5]:
if __name__ == "__main__":
    env = gym.make("Pong-v0")
    state = env.reset()
    prev_x = None
    score = 0
    episode = 0

    state_size = 80 * 80
    action_size = env.action_space.n
    agent = PGAgent(state_size, action_size)
    
agent.summary();

[2017-06-13 13:55:55,967] Making new env: Pong-v0


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 80, 80, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 27, 27, 25)        925       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 27, 27, 5)         4505      
_________________________________________________________________
flatten_2 (Flatten)          (None, 3645)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 20)                72920     
_________________________________________________________________
dense_5 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 126       
Total para

/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(25, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(3, 3))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(1, 1))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:30: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="he_uniform", activation="relu")`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="he_uniform", activation="relu")`


In [6]:
if __name__ == "__main__":
    env = gym.make("Pong-v0")
    state = env.reset()
    prev_x = None
    score = 0
    episode = 0

    state_size = 80 * 80
    action_size = env.action_space.n
    agent = PGAgent(state_size, action_size)
#     agent.summary();

AgentName = 'pong_minimal-s2'
AgentFile = 'Models/%s.h5'%AgentName;
LogName = 'Models/%s.log'%AgentName;
# LogFile = open(LogName,'a');
resume = 1;
render = 0;
agent.readlog(LogName)
print(agent.episode)

[2017-06-13 13:55:56,496] Making new env: Pong-v0


1141


/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(25, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(3, 3))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(1, 1))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:30: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="he_uniform", activation="relu")`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="he_uniform", activation="relu")`


In [7]:
if __name__ == "__main__":
    env = gym.make("Pong-v0")
    state = env.reset()
    prev_x = None
    score = 0
    episode = 0

    state_size = 80 * 80
    action_size = env.action_space.n
    agent = PGAgent(state_size, action_size)
#     agent.summary();
    

    if resume:
        agent.load(AgentFile);
        agent.readlog(LogName);
        episode = agent.episode;
    while episode > -1:
        if render:
            env.render()

        cur_x = preprocess(state)
        x = cur_x - prev_x if prev_x is not None else np.zeros(state_size)
        prev_x = cur_x

        action, prob = agent.act(np.expand_dims(x,1).T)
        state, reward, done, info = env.step(action)
        score += reward
        agent.remember(x, action, prob, reward)
        
        if done:
            episode += 1
            rewards = np.vstack(agent.rewards)
            rewards = agent.discount_rewards(rewards)
            agent.train(rewards)
            msg = '%s\t%d\t%s\t%f' % ('Episode',episode,'Score', score);
            print(msg);
            with open(LogName,'a+') as LogFile:
                LogFile.write(msg+'\n');
            
            score = 0
            state = env.reset()
            prev_x = None
            if episode > 1 and episode % 10 == 0:
                agent.save(AgentFile)

[2017-06-13 13:55:57,049] Making new env: Pong-v0
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(25, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(3, 3))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(1, 1))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:30: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="he_uniform", activation="relu")`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="he_uniform", activation="relu")`


Episode	1142	Score	-6.000000
Episode	1143	Score	-11.000000
Episode	1144	Score	-15.000000
Episode	1145	Score	-14.000000
Episode	1146	Score	-14.000000
Episode	1147	Score	-5.000000
Episode	1148	Score	-9.000000
Episode	1149	Score	-11.000000
Episode	1150	Score	-12.000000
Episode	1151	Score	-16.000000
Episode	1152	Score	-8.000000
Episode	1153	Score	-9.000000
Episode	1154	Score	-14.000000
Episode	1155	Score	-13.000000
Episode	1156	Score	-11.000000
Episode	1157	Score	-8.000000
Episode	1158	Score	-11.000000
Episode	1159	Score	-2.000000
Episode	1160	Score	-11.000000
Episode	1161	Score	-10.000000
Episode	1162	Score	-12.000000
Episode	1163	Score	-10.000000
Episode	1164	Score	-9.000000
Episode	1165	Score	-9.000000
Episode	1166	Score	-13.000000
Episode	1167	Score	-5.000000
Episode	1168	Score	-9.000000
Episode	1169	Score	-13.000000
Episode	1170	Score	-11.000000
Episode	1171	Score	-7.000000
Episode	1172	Score	-9.000000
Episode	1173	Score	-10.000000
Episode	1174	Score	-13.000000
Episode	1175	Score	-11.

Episode	1421	Score	-13.000000
Episode	1422	Score	-4.000000
Episode	1423	Score	-14.000000
Episode	1424	Score	-14.000000
Episode	1425	Score	-16.000000
Episode	1426	Score	-14.000000
Episode	1427	Score	-10.000000
Episode	1428	Score	-9.000000
Episode	1429	Score	-4.000000
Episode	1430	Score	-6.000000
Episode	1431	Score	-4.000000
Episode	1432	Score	-5.000000
Episode	1433	Score	-5.000000
Episode	1434	Score	-2.000000
Episode	1435	Score	-5.000000
Episode	1436	Score	-10.000000
Episode	1437	Score	-11.000000
Episode	1438	Score	-4.000000
Episode	1439	Score	-7.000000
Episode	1440	Score	-11.000000
Episode	1441	Score	-11.000000
Episode	1442	Score	-5.000000
Episode	1443	Score	-10.000000
Episode	1444	Score	-5.000000
Episode	1445	Score	-11.000000
Episode	1446	Score	-8.000000
Episode	1447	Score	-10.000000
Episode	1448	Score	-7.000000
Episode	1449	Score	-5.000000
Episode	1450	Score	-8.000000
Episode	1451	Score	-10.000000
Episode	1452	Score	-8.000000
Episode	1453	Score	-15.000000
Episode	1454	Score	-4.000000

Episode	1702	Score	-9.000000
Episode	1703	Score	-10.000000
Episode	1704	Score	-6.000000
Episode	1705	Score	-8.000000
Episode	1706	Score	-7.000000
Episode	1707	Score	-11.000000
Episode	1708	Score	-13.000000
Episode	1709	Score	-9.000000
Episode	1710	Score	-10.000000
Episode	1711	Score	-10.000000
Episode	1712	Score	-9.000000
Episode	1713	Score	-6.000000
Episode	1714	Score	-8.000000
Episode	1715	Score	-14.000000
Episode	1716	Score	-13.000000
Episode	1717	Score	-11.000000
Episode	1718	Score	-11.000000
Episode	1719	Score	-10.000000
Episode	1720	Score	-12.000000
Episode	1721	Score	-12.000000
Episode	1722	Score	-11.000000
Episode	1723	Score	-11.000000
Episode	1724	Score	-12.000000
Episode	1725	Score	-8.000000
Episode	1726	Score	-12.000000
Episode	1727	Score	-7.000000
Episode	1728	Score	-15.000000
Episode	1729	Score	-11.000000
Episode	1730	Score	-5.000000
Episode	1731	Score	-4.000000
Episode	1732	Score	-11.000000
Episode	1733	Score	-11.000000
Episode	1734	Score	-3.000000
Episode	1735	Score	-9.0

Episode	1981	Score	-10.000000
Episode	1982	Score	-11.000000
Episode	1983	Score	-8.000000
Episode	1984	Score	-11.000000
Episode	1985	Score	-10.000000


KeyboardInterrupt: 